# Final Project - Transfer Learning with VGG16
---

In [ ]:
# ===== TASK 1: Print the version of TensorFlow =====

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os, warnings
warnings.filterwarnings('ignore')

print("TensorFlow Version:", tf.__version__)

In [ ]:
# Set paths and hyperparameters

train_dir      = 'data/train'
validation_dir = 'data/validation'
test_dir       = 'data/test'

IMG_HEIGHT  = 224
IMG_WIDTH   = 224
BATCH_SIZE  = 32
NUM_CLASSES = len(os.listdir(train_dir))
EPOCHS      = 10

print(f"Number of classes: {NUM_CLASSES}")

In [ ]:
# Data Generators

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen       = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# ===== TASK 2: Create test_generator using test_datagen =====

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
print("test_generator created successfully.")

In [ ]:
# ===== TASK 3: Print the length of train_generator =====

print("Length of train_generator:", len(train_generator))

In [ ]:
# Build Extract-Features Model (VGG16 frozen)

base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

extract_feat_model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# ===== TASK 4: Print the summary of the model =====

extract_feat_model.summary()

In [ ]:
# ===== TASK 5: Compile the model =====

extract_feat_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print("Model compiled successfully.")

In [ ]:
# Train Extract-Features Model

extract_feat_history = extract_feat_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

In [ ]:
# ===== TASK 6: Plot accuracy curves for training and validation (extract_feat_model) =====

plt.figure(figsize=(8, 5))
plt.plot(extract_feat_history.history['accuracy'],    label='Training Accuracy')
plt.plot(extract_feat_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Extract Features Model — Accuracy Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Build & Train Fine-Tuned Model

fine_tune_model = tf.keras.models.clone_model(extract_feat_model)
fine_tune_model.set_weights(extract_feat_model.get_weights())

for layer in fine_tune_model.layers[:-4]:
    layer.trainable = False
for layer in fine_tune_model.layers[-4:]:
    layer.trainable = True

fine_tune_model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

fine_tune_history = fine_tune_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

In [ ]:
# ===== TASK 7: Plot loss curves for training and validation (fine_tune_model) =====

plt.figure(figsize=(8, 5))
plt.plot(fine_tune_history.history['loss'],     label='Training Loss')
plt.plot(fine_tune_history.history['val_loss'],  label='Validation Loss')
plt.title('Fine-Tuned Model — Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ===== TASK 8: Plot accuracy curves for training and validation (fine_tune_model) =====

plt.figure(figsize=(8, 5))
plt.plot(fine_tune_history.history['accuracy'],    label='Training Accuracy')
plt.plot(fine_tune_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Fine-Tuned Model — Accuracy Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Helper function: predict & plot a single test image

class_labels = list(train_generator.class_indices.keys())

def plot_test_image(model, model_name, index_to_plot=1):
    test_generator.reset()
    imgs, labels = next(test_generator)
    while index_to_plot >= len(imgs):
        more_imgs, more_labels = next(test_generator)
        imgs   = np.concatenate([imgs,   more_imgs])
        labels = np.concatenate([labels, more_labels])

    img   = imgs[index_to_plot]
    label = labels[index_to_plot]

    pred       = model.predict(np.expand_dims(img, axis=0))
    pred_class = class_labels[np.argmax(pred)]
    true_class = class_labels[np.argmax(label)]

    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    plt.title(f"{model_name}\nTrue: {true_class}  |  Predicted: {pred_class}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# ===== TASK 9: Plot a test image using Extract Features Model (index_to_plot = 1) =====

plot_test_image(extract_feat_model,
                model_name='Extract Features Model',
                index_to_plot=1)

In [ ]:
# ===== TASK 10: Plot a test image using Fine-Tuned Model (index_to_plot = 1) =====

plot_test_image(fine_tune_model,
                model_name='Fine-Tuned Model',
                index_to_plot=1)